# Meter los datos en la base de datos

Puedes ejecutar este notebook desde la linea de comandos con:

```bash
papermill --cwd ./chat -k python3 --no-log-output chat/add_info_python.ipynb /tmp/output.log.json
```

In [10]:
import os
from tqdm.notebook import tqdm
import more_itertools
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

print(f"Inicializando chromadb")
NUM_CHUNKS = 7
MODEL = "llama3"

chroma_client = chromadb.PersistentClient(path='db') # current folder
llama3_embedding_function = embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings", # ollama running in the background
    model_name=MODEL,
)

collection_glossary = chroma_client.get_or_create_collection(name="glossary",
                                                      embedding_function=llama3_embedding_function)
collection_turras = chroma_client.get_or_create_collection(name="turras",
                                                    embedding_function=llama3_embedding_function)


Inicializando chromadb


# Meter términos de glosario


In [2]:
import csv
print("Glosario loader")
def glosario():
    file_path = os.path.join(os.getcwd(), '..', 'db', 'glosario.csv')
    
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, fieldnames=['word', 'definition', 'sources'], delimiter=',', quotechar='"')
        next(reader) # header
        for row in reader:
            yield {
                'word': row['word'],
                'definition': row['definition'],
                'sources': row['sources'] or ''
            }

glos_documents = []
glos_ids = []

for item in glosario():
    glos_documents.append(f"{item['word']}: {item['definition']}")
    glos_ids.append(item['word'])
    
ids=list(more_itertools.chunked(glos_documents, NUM_CHUNKS))
docs=list(more_itertools.chunked(glos_ids, NUM_CHUNKS))
for ids, docs in tqdm(list(zip(ids, docs))):
    collection_glossary.upsert(documents=glos_documents, ids=glos_ids)

Glosario loader


  0%|          | 0/9 [00:00<?, ?it/s]

## Meter las turras

In [3]:
import json
from pprint import pprint
import os
from tqdm import tqdm

print(f"Turra loader")
NUM_CHUNKS = 7

def turras():
    urls = []
    completas = []

    with open(os.path.join(os.getcwd(), '..', 'db', 'tweets.json'), mode='r', encoding='utf-8') as jsonfile:
        turras = json.load(jsonfile)
        for turra in turras:
            try:
                i = str(turra[0]['id'])
                urls.append("https://turrero.vercel.app/turra/" + i)
                completas.append('\n'.join(tuit["tweet"] for tuit in turra))
            except Exception as e:
                pprint(turra)
                raise e
    assert len(urls) == len(completas), "Bug: no hay la misma cantidad de urls y turras"
    return (urls, completas)

# Partirlo en cachos para poder ver la barrita llenarse, pues tarda mucho
urls_list, turras_list = turras()

urls_chunks = list(more_itertools.chunked(urls_list, NUM_CHUNKS))
turras_chunks = list(more_itertools.chunked(turras_list, NUM_CHUNKS))
batched = list(zip(urls_chunks, turras_chunks))

for ids, docs in tqdm(batched):
    collection_turras.upsert(
        documents=docs,
        ids=ids
    )

Turra loader


100%|██████████| 27/27 [1:08:36<00:00, 152.47s/it]


# Ejemplo de Query

In [53]:
turras = collection_turras.query(query_texts=["¿qué es la estrategia?"], n_results=3)
turras

{'ids': [['https://turrero.vercel.app/turra/967390304260239360',
   'https://turrero.vercel.app/turra/1801852274232218084',
   'https://turrero.vercel.app/turra/1297462657503694848']],
 'distances': [[15741.848967303671, 17606.33867662567, 17747.184963512907]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['Hilo imprescindible sobre que es lo que importa en una startup de verdad. Propuesta de valor Customer centric y narrativa identificable.',
   'En el hilo turras de hoy, y continuando con la serie de sensemaking, hablaremos de una de las tareas de Hércules: Hacer cambiar a alguien de opinión.\nEl meme “Change My Mind” se originó a partir de una fotografía del podcaster conservador estadounidense-canadiense Steven Crowder.\nEn el aparece sentado detrás de una mesa con un cartel que decía “Male Privilege is a myth / Change My Mind” (El privilegio masculino es un mito / Hazme cambiar de opinión) fuera de la Universidad Cristiana de Texas.\nCrowder publicó la i

In [52]:
definiciones = collection_glossary.query(query_texts=["¿qué es la estrategia?"], n_results=5)
definiciones

{'ids': [['Sensemaking',
   'Suerte preparada',
   'Suerte buscada',
   'Cinco Porqués (5 Whys)',
   'Business Acumen']],
 'distances': [[12452.928302049551,
   12562.02687845355,
   14036.732288106916,
   14227.070883154713,
   14289.630540898685]],
 'metadatas': [[None, None, None, None, None]],
 'embeddings': None,
 'documents': [['Sensemaking: Interpretar y entender situaciones complejas o inciertas para tomar decisiones.',
   'Suerte preparada: Aprovechar oportunidades gracias a la preparación previa.',
   'Suerte buscada: Actuar proactivamente para crear oportunidades de éxito.',
   'Cinco Porqués (5 Whys): desentrañar la cadena de causas que llevan a un problema específico.',
   'Business Acumen: Uno de los 4 pilares del CPS: conocimiento e intuición para los negocios, normalmente combinación de intuición, experiencia y conocimiento cristalizado (ej. conocimiento académico).']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [11]:
import ollama



stream = ollama.chat(
    model=MODEL,
    messages=[{'role': 'user', 'content': '¿qué es la estrategia?'}],
    stream=True,
)


for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

La estrategia es un plan o conjunto de acciones que se diseña y ejecuta para alcanzar objetivos específicos, superar desafíos o mejorar situaciones. En general, una estrategia implica una serie de decisiones y acciones coordinadas que se toman con el fin de lograr un objetivo particular.

La estrategia puede ser aplicada en diferentes áreas, como la empresa, la política, la educación, la salud, etc. En el contexto empresarial, una estrategia es un plan detallado para alcanzar objetivos comerciales y competitividad, que se basa en analizar las fortalezas, debilidades, oportunidades y amenazas (análisis FODA) de la empresa.

Las características generales de una buena estrategia son:

1. Claridad: La estrategia debe ser clara y fácilmente entendible para todos los involucrados.
2. Realismo: La estrategia debe ser realista y factible de implementar.
3. Priorización: La estrategia debe priorizar los objetivos más importantes y enfocarse en las acciones que más impacto tendrán.
4. Flexibilid

In [3]:
prompt = """

Sigue las siguientes instrucciones. Eres un asistente para tareas de tipo 'pregunta y respuesta', solo puedes dar respuestas cortas de máximo un párrafo. Utiliza la información del contexto para crer una respuesta corta, breve y en registro oral, pero no te alargues ni incluyas toda la información del contexto si no es necesario.

Pregunta del usuario: {question} 
Contexto de la conversación: {context} 
Respuesta (en su mismo idioma):
"""

In [2]:
question = '¿qué es la estrategia?'
context = '\n'.join(definiciones['documents'][0] + turras['documents'][0])
full_prompt = prompt.format(question=question, context=context)
len(full_prompt)

full_prompt

NameError: name 'definiciones' is not defined

In [56]:
stream = ollama.chat(
    model=MODEL,
    messages=[{'role': 'user', 'content': full_prompt}],
    stream=True,
)


for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

¡Hola! Como asistente, mi objetivo es ayudarte a seguir las instrucciones y aclarar cualquier duda. En este caso, parece que estás hablando sobre el diseño y el Design Thinking en general.

Me gustó cómo definiste la liminalidad como un estado crítico en el que se produce la innovación, cuando personas con diferentes disciplinas se encuentran y se producen fricciones entre ellas.

También me llamó la atención la forma en que describías el papel de Steve Jobs en Apple, al crear una fricción liminar entre el diseño y la tecnología. Eso fue un momento clave para la empresa y su éxito.

En cuanto a tu problema histórico con el Design Thinking, parece que estás preocupado por cómo algunas personas lo han implementado de manera incorrecta, como si fuera un plato maestro sin un cocinero liminal que lo entienda.

Me gustaría saber más sobre tus ideas y experiencia en este campo. ¿Qué te parece si seguimos explorando la relación entre el diseño, la tecnología y la innovación?